In [1]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html
from dash.dependencies import Input, Output

In [13]:
data = pd.read_excel('Llamados2024.xlsx')
# mostrar los primeros 5 registros
print(data.head())

       fecha      hora   prevision      estado transaccion  \
0 2024-01-02  10:05:00      Fonasa  finalizada          si   
1 2024-01-02  10:20:00      Fonasa  contactada          no   
2 2024-01-02  12:00:00      Fonasa  finalizada          si   
3 2024-01-02  18:00:00  Particular     anulada          no   
4 2024-01-02  17:00:00      Fonasa     anulada          no   

         tipoconsulta   rutMedico           Medico  rutDadopor  \
0  Consulta sin Costo  16690652-0     DRA.TGardner  16690652-0   
1   Control con Costo   5954756-9      DRA.RAlbert   5954756-9   
2  Consulta con Costo  16690652-0     DRA.TGardner  16690652-0   
3  Consulta sin Costo  14985586-6  DRA.AMccullough  14985586-6   
4   Control sin Costo  24266756-5      DRA.KDorsey  24266756-5   

           dadopor dadoporperfil rutcontactadopor contactadopor  \
0     DRA.TGardner        MEDICO       17042022-5        EPerry   
1      DRA.RAlbert        MEDICO       17042022-5        EPerry   
2     DRA.TGardner        MED

1. Administrativos v/s Cantidad de pacientes contactados
• En el estado inicial, sin filtros asignados, deberá mostrar el conteo ordenado de menor a mayor.
• Este gráfico deberá permitir seleccionar, respecto a la cantidad, entre 3 categorías: Entre 0 a 4000, Entre
4001 a 8000 y Entre 8001 en adelante.
• Adicionalmente deberá existir un control que permitirá seleccionar el nombre de un administrativo, el que al
activarse hará una comparación de él frente al promedio.

2. Médicos v/s Horas Anuladas
• En el estado inicial, sin filtros asignados, deberá mostrar el conteo de los promedios de cada médico,
ordenado de menor a mayor.
• Los controles por considerar son: Listado de Médicos (incluir un ítem “todos”), Dado Por (Web, Médico o
Administrativo) y Meses. El control de Dado Por deberá filtrar la información de horas anuladas, pero que
fueron “dadas por” el tipo de personal seleccionado.
• En este gráfico los controles podrán ser “complementarios”, por lo que, por ejemplo, podría visualizar el
listado de las horas anuladas que fueron dadas por un Administrativo en el mes de Julio.

3. Horas Finalizadas v/s Tipo Consulta
• En el estado inicial, sin filtros asignados, deberá mostrar el conteo de horas finalizadas por cada Tipo de
Consulta, representando lo “%sin costo” en un color distinto a lo “%con costo”.
• Se deberá tener un control para los distintos “Tipos de Consulta” y adicionalmente uno para el “mes”, por
defecto considera todos los tipos y todos los meses, a menos que el usuario seleccione algo distinto.


In [5]:
# Filter data to get the count of patients contacted by each administrative
contacted_counts = data['contactadopor'].value_counts().reset_index()
contacted_counts.columns = ['contactadopor', 'count']
contacted_counts = contacted_counts.sort_values(by='count')


In [9]:

# Filter data to get the count of canceled hours by each doctor
canceled_hours = data[data['estado'] == 'anulada']
canceled_hours_counts = canceled_hours['Medico'].value_counts().reset_index()
canceled_hours_counts.columns = ['Medico', 'count']
canceled_hours_counts = canceled_hours_counts.sort_values(by='count')

avg_canceled_hours = canceled_hours_counts['count'].mean()

In [11]:
# Initialize the Dash app
# Calculate the average canceled hours per doctor
avg_canceled_hours = canceled_hours_counts['count'].mean()

# Calculate the count of finalized hours by type of consultation
finalized_hours = data[data['estado'] == 'finalizada']
finalized_hours_counts = finalized_hours['tipoconsulta'].value_counts().reset_index()
finalized_hours_counts.columns = ['tipoconsulta', 'count']

app = Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard"),
    html.Div([
        html.H2("Administrativos v/s Cantidad de pacientes contactados"),
        dcc.Dropdown(
            id='category-dropdown',
            options=[
                {'label': 'Sin filtro', 'value': 'all'},
                {'label': 'Entre 0 a 4000', 'value': '0-4000'},
                {'label': 'Entre 4001 a 8000', 'value': '4001-8000'},
                {'label': 'Entre 8001 en adelante', 'value': '8001+'}
            ],
            value='all'
        ),
        dcc.Dropdown(
            id='admin-dropdown',
            options=[{'label': admin, 'value': admin} for admin in contacted_counts['contactadopor']],
            placeholder="Select an administrative"
        ),
        dcc.Graph(id='bar-graph')
    ]),
    html.Div([
        html.H2("Médicos v/s Horas Anuladas"),
        dcc.Dropdown(
            id='medico-dropdown',
            options=[{'label': 'Todos', 'value': 'all'}] + [{'label': medico, 'value': medico} for medico in canceled_hours_counts['Medico']],
            value='all'
        ),
        dcc.Dropdown(
            id='dado-por-dropdown',
            options=[
                {'label': 'Web', 'value': 'Web'},
                {'label': 'Médico', 'value': 'MEDICO'},
                {'label': 'Administrativo', 'value': 'ADMINISTRATIVO'}
            ],
            placeholder="Select Dado Por"
        ),
        dcc.Dropdown(
            id='mes-dropdown',
            options=[{'label': mes, 'value': mes} for mes in data['fecha'].dt.strftime('%Y-%m').unique()],
            placeholder="Select a month"
        ),
        dcc.Graph(id='canceled-hours-graph')
    ]),
    html.Div([
        html.H2("Horas Finalizadas v/s Tipo Consulta"),
        dcc.Dropdown(
            id='tipo-consulta-dropdown',
            options=[{'label': tipo, 'value': tipo} for tipo in finalized_hours_counts['tipoconsulta']],
            placeholder="Select Tipo Consulta"
        ),
        dcc.Dropdown(
            id='mes-finalizada-dropdown',
            options=[{'label': mes, 'value': mes} for mes in data['fecha'].dt.strftime('%Y-%m').unique()],
            placeholder="Select a month"
        ),
        dcc.Graph(id='finalized-hours-graph')
    ])
])

@app.callback(
    Output('bar-graph', 'figure'),
    [Input('category-dropdown', 'value'),
     Input('admin-dropdown', 'value')]
)
def update_admin_graph(selected_category, selected_admin):
    if selected_category == '0-4000':
        filtered_data = contacted_counts[contacted_counts['count'] <= 4000]
    elif selected_category == '4001-8000':
        filtered_data = contacted_counts[(contacted_counts['count'] > 4000) & (contacted_counts['count'] <= 8000)]
    elif selected_category == '8001+':
        filtered_data = contacted_counts[contacted_counts['count'] > 8000]
    else:
        filtered_data = contacted_counts

    fig = px.bar(filtered_data, x='contactadopor', y='count', title='Cantidad de pacientes contactados')

    if selected_admin:
        admin_count = contacted_counts[contacted_counts['contactadopor'] == selected_admin]['count'].values[0]
        avg_count = contacted_counts['count'].mean()
        fig.add_trace(px.bar(x=[selected_admin, 'Promedio'], y=[admin_count, avg_count], title='Comparación con el promedio').data[0])

    return fig

@app.callback(
    Output('canceled-hours-graph', 'figure'),
    [Input('medico-dropdown', 'value'),
     Input('dado-por-dropdown', 'value'),
     Input('mes-dropdown', 'value')]
)
def update_canceled_hours_graph(selected_medico, selected_dado_por, selected_mes):
    filtered_data = canceled_hours
    if selected_medico != 'all':
        filtered_data = filtered_data[filtered_data['Medico'] == selected_medico]
    if selected_dado_por:
        filtered_data = filtered_data[filtered_data['dadoporperfil'] == selected_dado_por]
    if selected_mes:
        filtered_data = filtered_data[filtered_data['fecha'].dt.strftime('%Y-%m') == selected_mes]

    canceled_hours_counts = filtered_data['Medico'].value_counts().reset_index()
    canceled_hours_counts.columns = ['Medico', 'count']
    canceled_hours_counts = canceled_hours_counts.sort_values(by='count')

    fig = px.bar(canceled_hours_counts, x='Medico', y='count', title='Horas Anuladas por Médico')

    return fig

@app.callback(
    Output('finalized-hours-graph', 'figure'),
    [Input('tipo-consulta-dropdown', 'value'),
     Input('mes-finalizada-dropdown', 'value')]
)
def update_finalized_hours_graph(selected_tipo_consulta, selected_mes):
    filtered_data = finalized_hours
    if selected_tipo_consulta:
        filtered_data = filtered_data[filtered_data['tipoconsulta'] == selected_tipo_consulta]
    if selected_mes:
        filtered_data = filtered_data[filtered_data['fecha'].dt.strftime('%Y-%m') == selected_mes]

    finalized_hours_counts = filtered_data['tipoconsulta'].value_counts().reset_index()
    finalized_hours_counts.columns = ['tipoconsulta', 'count']

    fig = px.bar(finalized_hours_counts, x='tipoconsulta', y='count', title='Horas Finalizadas por Tipo de Consulta')

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)